In [11]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, InputLayer, Dropout
from tensorflow.keras import Sequential
from tensorflow.keras.optimizers import Adam
from imblearn.over_sampling import SMOTE

In [12]:
TRAIN_DIR = '../../data/train'
TEST_DIR = '../../data/test'
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
EPOCHS = 10

In [13]:
from tensorflow.keras.applications.resnet50 import preprocess_input

train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

train_generator = train_datagen.flow_from_directory(
    TRAIN_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False  # Turn off shuffle for consistent label ordering
)

test_generator = test_datagen.flow_from_directory(
    TEST_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

num_train_samples = train_generator.samples
num_test_samples = test_generator.samples
num_classes = train_generator.num_classes

print(f"Train samples: {num_train_samples}, Test samples: {num_test_samples}, Classes: {num_classes}")


Found 5181 images belonging to 26 classes.
Found 1307 images belonging to 26 classes.
Train samples: 5181, Test samples: 1307, Classes: 26


In [14]:
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
pool_layer = GlobalAveragePooling2D()

feature_extractor = Sequential([
    base_model,
    pool_layer
], name="FeatureExtractor")

feature_extractor.summary()

Model: "FeatureExtractor"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ resnet50 (Functional)           │ (None, 7, 7, 2048)     │    23,587,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_1      │ (None, 2048)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 23,587,712 (89.98 MB)

 Trainable params: 23,534,592 (89.78 MB)

 Non-trainable params: 53,120 (207.50 KB)

In [15]:

print("Extracting training embeddings...")
X_train_embed_list = []
y_train_list = []

steps_train = len(train_generator)  # steps in one epoch for training data
for i in range(steps_train):
    X_batch, y_batch = train_generator[i]  # (batch_size, 224, 224, 3)
    embeddings = feature_extractor.predict(X_batch)  # (batch_size, 2048) typically for ResNet50
    X_train_embed_list.append(embeddings)
    y_train_list.append(y_batch)

X_train_embed = np.concatenate(X_train_embed_list, axis=0)  # shape: (num_train_samples, 2048)
y_train = np.concatenate(y_train_list, axis=0)             # shape: (num_train_samples, num_classes)

# Convert one-hot to integer labels
y_train_int = np.argmax(y_train, axis=1)

print("X_train_embed shape:", X_train_embed.shape)
print("y_train_int shape:", y_train_int.shape)

Extracting training embeddings...
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 613ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 651ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 587ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 628ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 593ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 627ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 625ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 586ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 614ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 637ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 631ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 598ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 589ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 626ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 639ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 631ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 632ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 623ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 622ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 609ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 615ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 613ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 626ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 667ms

In [16]:
smote = SMOTE(random_state=42)
X_train_sm, y_train_sm = smote.fit_resample(X_train_embed, y_train_int)

print("After SMOTE:")
print("X_train_sm shape:", X_train_sm.shape)
print("y_train_sm shape:", y_train_sm.shape)

# Convert back to one-hot
y_train_sm_onehot = tf.keras.utils.to_categorical(y_train_sm, num_classes=num_classes)

After SMOTE:
X_train_sm shape: (11960, 2048)
y_train_sm shape: (11960,)


In [17]:
classifier = Sequential([
    InputLayer(input_shape=(X_train_sm.shape[1],)),  # e.g., 2048 for ResNet50
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')
])

classifier.compile(
    optimizer=Adam(learning_rate=1e-3),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

classifier.summary()

c:\Users\tobia\conda\miniconda3\Lib\site-packages\keras\src\layers\core\input_layer.py:27: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_2 (Dense)                 │ (None, 256)            │       524,544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 26)             │         6,682 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 531,226 (2.03 MB)

 Trainable params: 531,226 (2.03 MB)

 Non-trainable params: 0 (0.00 B)

In [18]:
history = classifier.fit(
    X_train_sm,
    y_train_sm_onehot,
    epochs=EPOCHS,
    validation_split=0.2,
    verbose=1
)

Epoch 1/10


299/299 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.4102 - loss: 2.2384 - val_accuracy: 0.5920 - val_loss: 1.3232
Epoch 2/10
299/299 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8026 - loss: 0.6608 - val_accuracy: 0.8365 - val_loss: 0.5600
Epoch 3/10
299/299 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8744 - loss: 0.4154 - val_accuracy: 0.8599 - val_loss: 0.4412
Epoch 4/10
299/299 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8965 - loss: 0.3317 - val_accuracy: 0.9377 - val_loss: 0.2921
Epoch 5/10
299/299 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9162 - loss: 0.2709 - val_accuracy: 0.9674 - val_loss: 0.1877
Epoch 6/10
299/299 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9287 - loss: 0.2225 - val_accuracy: 0.9427 - val_loss: 0.2060
Epoch 7/10
299/299 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9392 - loss: 0.1834 - val_accuracy: 0.9770 - val_loss: 0.1435
Epoch 8/10
299/299 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9344 - loss: 0.1971 - val_accuracy: 0.9624 - val_

In [19]:
print("\nExtracting test embeddings...")
X_test_embed_list = []
y_test_list = []

steps_test = len(test_generator)
for i in range(steps_test):
    X_batch, y_batch = test_generator[i]
    embeddings = feature_extractor.predict(X_batch)
    X_test_embed_list.append(embeddings)
    y_test_list.append(y_batch)

X_test_embed = np.concatenate(X_test_embed_list, axis=0)
y_test = np.concatenate(y_test_list, axis=0)
y_test_int = np.argmax(y_test, axis=1)

print("X_test_embed shape:", X_test_embed.shape)
print("y_test_int shape:", y_test_int.shape)

# Evaluate classifier
test_probs = classifier.predict(X_test_embed)
test_preds = np.argmax(test_probs, axis=1)

accuracy = np.mean(test_preds == y_test_int)
print(f"Test Accuracy: {accuracy:.3f}")

# You could compute other metrics if desired
from sklearn.metrics import classification_report
print("\nClassification Report:")
print(classification_report(y_test_int, test_preds))


Extracting test embeddings...


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 611ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 592ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 582ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 592ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 588ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 651ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 600ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 598ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 591ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 598ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 574ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 597ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 581ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 575ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 590ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 573ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 587ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 597ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 589ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 586ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 580ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 584ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 608ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 590ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 582ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 

In [20]:
classifier.save('smote_classifier.h5')
print("Classifier saved to smote_classifier.h5")

Classifier saved to smote_classifier.h5
